In [ ]:
import pandas as pd
from datetime import date, datetime
output_data = pd.DataFrame(columns=["number_of_teams"])

In [ ]:
def get_number_of_teams(file_contents: str) -> int:
    EXCLUDE_REPOS = {"dapla-felles-iac", "demo-enhjoern-a-iac", "demo-enhjoern-b-iac"}
    line = [line.strip() for line in file_contents.splitlines() if line.strip().startswith("value: github.com")][0]
    sections = [l for l in line.split(",") if not(any([r in l for r in EXCLUDE_REPOS]))]
    return len(sections)

In [ ]:
# Assumes https://github.com/statisticsnorway/atlantis-team-config is cloned to the home directory
from git import Repo, GitCommandError
from pathlib import Path
from pprint import pprint
PATH_TO_CONFIG_FILE = "kubernetes-manifests/dapla-felles/atlantis-statefulset/atlantis-statefulset.yaml"
atlantis_config = Repo(Path.home() / "atlantis-team-config")
main = atlantis_config.heads.main
for commit in atlantis_config.iter_commits("main", max_count=200):
    # Get contents of file for this commit
    try:
        file_contents = atlantis_config.git.show('{}:{}'.format(commit.hexsha, PATH_TO_CONFIG_FILE))
    except GitCommandError:
        break
    number_of_teams = get_number_of_teams(file_contents)
    commit_datetime = commit.committed_datetime
    output_data.at[commit_datetime.date(), "number_of_teams"] = number_of_teams

In [ ]:
import plotly
import plotly.io as pio
import plotly.express as px
import nbformat

pd.options.plotting.backend = "plotly"

output_data.sort_index(inplace=True)
output_data = output_data.truncate(before=date(2022, 3, 15), after=datetime.now().date())
figure = px.line(output_data, title="Team etablert på Dapla", template="plotly_dark",
              labels=dict(index="Dato", value="Antall team", variable="Antall"))
figure.update_layout(showlegend=False)
figure.show()